In [1]:
import tensorflow_datasets as tfds

In [2]:
import tensorflow_datasets as tfds

# Load the IMDb reviews dataset
# This will download the data the first time you run it
(train_data, test_data), info = tfds.load(
    'imdb_reviews',
    split=('train', 'test'),
    with_info=True,
    as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/incomplete.WKZY7B_1.0.0/imdb_reviews-train.tfrecor…

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/incomplete.WKZY7B_1.0.0/imdb_reviews-test.tfrecord…

Generating unsupervised examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/incomplete.WKZY7B_1.0.0/imdb_reviews-unsupervised.…

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [3]:
# Let's see what the data looks like
for review, label in train_data.take(2):
  print("Review:", review.numpy().decode('utf-8'))
  print("Label:", label.numpy()) # 0 is negative, 1 is positive
  print("-" * 20)

Review: This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.
Label: 0
--------------------
Review: I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell asleep because the f

In [5]:
import pandas as pd
import numpy as np

# The 'as_dataframe' function failed. We will do it manually.
# We will loop through the dataset and pull out every review and label
# one by one and put them into Python lists.
# This will take a moment.

print("Manually extracting training data...")
train_reviews = []
train_labels = []

# This loop goes through the 'train_data' pipe, grabs each (review, label) pair
for review, label in train_data:
    # .numpy() converts from a TensorFlow 'Tensor' object to a numpy value
    # .decode('utf-8') converts from bytes (b'...') to a normal string
    train_reviews.append(review.numpy().decode('utf-8'))
    train_labels.append(label.numpy())

print("Manually extracting testing data...")
test_reviews = []
test_labels = []

# Do the same for the test data
for review, label in test_data:
    test_reviews.append(review.numpy().decode('utf-8'))
    test_labels.append(label.numpy())

# Now, create the DataFrame (the "workbench table") from our lists.
# This is a reliable, standard way to build a DataFrame.
train_df = pd.DataFrame({
    'text': train_reviews,    # Column 1 is our list of review strings
    'label': train_labels     # Column 2 is our list of 0s and 1s
})

test_df = pd.DataFrame({
    'text': test_reviews,
    'label': test_labels
})

# Now, prove it worked. This is your sanity check.
print("\nTraining DataFrame Head (Manual Build):")
print(train_df.head())

Manually extracting training data...
Manually extracting testing data...

Training DataFrame Head (Manual Build):
                                                text  label
0  This was an absolutely terrible movie. Don't b...      0
1  I have been known to fall asleep during films,...      0
2  Mann photographs the Alberta Rocky Mountains i...      0
3  This is the kind of film for a snowy Sunday af...      1
4  As others have mentioned, all the women that g...      1


In [6]:
# Install the spaCy library and its small English model
# The '-q' makes the output less noisy.
!pip install -U spacy -q
!python -m spacy download en_core_web_sm -q


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 109.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [7]:
import spacy
import re

# Load the small English model.
# We disable the 'parser' and 'ner' components because we don't need them.
# We ONLY need the 'tagger' (for parts of speech) and 'lemmatizer'.
# This makes it MUCH faster.
print("Loading spaCy model...")
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
print("Model loaded.")

def clean_text(text):
    # 1. Remove HTML tags using a simple regex
    text = re.sub(r'<[^>]+>', '', text)

    # 2. Convert to lowercase
    text = text.lower()

    # 3. Create a spaCy 'doc' object.
    doc = nlp(text)

    # 4. Create a list of clean tokens (words)
    clean_tokens = []
    for token in doc:
        # We check two things:
        # 1. Is it NOT a stop word (like 'the', 'a', 'is')?
        # 2. Is it alphabetic (NOT punctuation, NOT a number)?
        if not token.is_stop and token.is_alpha:
            # If it passes, we get its 'lemma_' (the root form)
            clean_tokens.append(token.lemma_)

    # 5. Join the clean tokens back into a single string
    return ' '.join(clean_tokens)

Loading spaCy model...
Model loaded.


In [8]:
# WARNING: This will take several minutes. Be patient.
print("Cleaning training data (this will take a while)...")
train_df['clean_text'] = train_df['text'].apply(clean_text)

print("Cleaning testing data (this will also take a while)...")
test_df['clean_text'] = test_df['text'].apply(clean_text)

print("Cleaning complete.")

# Prove it worked. Show me the first review, raw vs. clean.
print("\n--- ORIGINAL REVIEW (Index 0) ---")
print(train_df['text'].iloc[0])
print("\n--- CLEANED REVIEW (Index 0) ---")
print(train_df['clean_text'].iloc[0])

Cleaning training data (this will take a while)...
Cleaning testing data (this will also take a while)...
Cleaning complete.

--- ORIGINAL REVIEW (Index 0) ---
This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.

--- CLEANED REVIEW (Index 0) ---
absolutely terrible movie lure christopher walken michael ironside great actor simply bad role h

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

# 1. Initialize the vectorizer.
# We'll limit it to the top 10,000 most frequent words.
# If we used every single word, your model would be pointlessly huge.
vectorizer = TfidfVectorizer(max_features=10000)

print("Vectorizing training data...")
# 2. FIT and TRANSFORM on the training data.
# This learns the vocabulary AND converts the text to numbers.
X_train_tfidf = vectorizer.fit_transform(train_df['clean_text'])

print("Vectorizing testing data...")
# 3. ONLY TRANSFORM on the testing data.
# We use the *same vocabulary* learned from the training data.
# This prevents 'data leakage' and is a critical step.
X_test_tfidf = vectorizer.transform(test_df['clean_text'])

# 4. Get the labels. These are already 0s and 1s, so they are ready.
y_train = train_df['label']
y_test = test_df['label']

# 5. Prove it worked. Show me the shape of your new data.
print("\n--- SHAPE OF TF-IDF MATRICES ---")
print("Train data shape:", X_train_tfidf.shape)
print("Test data shape:", X_test_tfidf.shape)
print("\n--- SHAPE OF LABELS ---")
print("Train labels shape:", y_train.shape)
print("Test labels shape:", y_test.shape)

Vectorizing training data...
Vectorizing testing data...

--- SHAPE OF TF-IDF MATRICES ---
Train data shape: (25000, 10000)
Test data shape: (25000, 10000)

--- SHAPE OF LABELS ---
Train labels shape: (25000,)
Test labels shape: (25000,)


In [10]:
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, confusion_matrix
import time

# --- STEP 4: Build and Train the Model ---

# 1. Initialize the model. We'll use the defaults.
# It's a workhorse. It doesn't need much tuning.
model = LinearSVC(dual=True) # dual=True is often faster for n_samples > n_features, but good to set. We'll stick with True for robustness.

print("Training the LinearSVC model...")

# 2. Train the model.
start_time = time.time()
model.fit(X_train_tfidf, y_train)
end_time = time.time()

print(f"--- Training finished in {end_time - start_time:.2f} seconds ---")


# --- STEP 5: Evaluate the Model (The RIGHT Way) ---

print("\nMaking predictions on the test data...")
# 1. Make predictions on the unseen test data.
y_pred = model.predict(X_test_tfidf)

# 2. Generate the Confusion Matrix.
# This shows you exactly where you were wrong.
print("\n--- Confusion Matrix ---")
print("   (Predicted Neg) (Predicted Pos)")
print(confusion_matrix(y_test, y_pred))

# 3. Generate the Classification Report.
# This is your main result. It shows precision, recall, and f1-score.
print("\n--- Classification Report ---")
print(classification_report(y_test, y_pred, target_names=['Negative (0)', 'Positive (1)']))

Training the LinearSVC model...
--- Training finished in 0.61 seconds ---

Making predictions on the test data...

--- Confusion Matrix ---
   (Predicted Neg) (Predicted Pos)
[[10806  1694]
 [ 2027 10473]]

--- Classification Report ---
              precision    recall  f1-score   support

Negative (0)       0.84      0.86      0.85     12500
Positive (1)       0.86      0.84      0.85     12500

    accuracy                           0.85     25000
   macro avg       0.85      0.85      0.85     25000
weighted avg       0.85      0.85      0.85     25000



In [11]:
print("--- NER Demonstration on Sample Reviews ---")

# We're not using the 'clean_text' function. NER needs real, raw text.
# We must load a model that *has* an NER component.
# nlp = spacy.load('en_core_web_sm') # If you disabled NER, load a fresh one

sample_reviews = [
    "I bought a new Sony PlayStation 5 from Amazon, and it's amazing.",
    "My old Samsung TV is better than this new Vizio one.",
    "The new album by U2 is not as good as their work in the 90s."
]

for review in sample_reviews:
    print(f"\nREVIEW: '{review}'")
    doc = nlp(review)

    if not doc.ents:
        print("  -> No entities found.")
    else:
        print("  -> ENTITIES FOUND:")
        for ent in doc.ents:
            # ent.text is the word, ent.label_ is the category
            print(f"    - {ent.text} ({ent.label_})")

--- NER Demonstration on Sample Reviews ---

REVIEW: 'I bought a new Sony PlayStation 5 from Amazon, and it's amazing.'
  -> No entities found.

REVIEW: 'My old Samsung TV is better than this new Vizio one.'
  -> No entities found.

REVIEW: 'The new album by U2 is not as good as their work in the 90s.'
  -> No entities found.
